In [ ]:
!pip install category_encoders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder 
from datetime import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import time
from sklearn.preprocessing import LabelEncoder
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def mergedata(file1,file2,file3):
  data3 = pd.read_csv(file1)
  data1 = pd.read_csv(file2) #name
  data2 = pd.read_csv(file3) #movie
  data1.rename(columns={'name': 'movie_title'}, inplace=True)
  data2.rename(columns={'movie': 'movie_title'}, inplace=True)
  data = pd.merge(data3,data1[['movie_title','director']],on='movie_title', how='left')
  data_actors= data2.groupby('movie_title')['voice-actor'].agg(list)
  my_data = pd.merge(data,data_actors,on='movie_title', how='left')
  my_data['release_date'] = pd.to_datetime(my_data['release_date'])
  my_data['release_date'] = pd.DatetimeIndex(my_data['release_date']).year
  for index, row in my_data.iterrows():
    if(row["release_date"] > 2022):
      my_data.at[index,"release_date"] = row["release_date"] - 100
  return my_data


In [ ]:
my_data = mergedata("movies-tas-classification-test.csv","movie-director.csv","movie-voice-actors.csv")

In [ ]:
my_data

,movie_title,release_date,genre,MPAA_rating,MovieSuccessLevel,director,voice-actor
0,Washington Square,1997,Drama,PG,D,NaN,NaN
1,Million Dollar Arm,2014,Drama,PG,C,NaN,NaN
2,The Rookie,2002,Drama,G,B,NaN,NaN
3,Betsy's Wedding,1990,Comedy,R,C,NaN,NaN
4,Tombstone,1993,Western,R,B,NaN,NaN
...,...,...,...,...,...,...,...
108,Innocent Blood,1992,Horror,R,D,NaN,NaN
109,One Good Cop,1991,Action,R,C,NaN,NaN
110,Doug's 1st Movie,1999,Adventure,G,C,NaN,NaN
111,Wild Hearts Can't Be Broken,1991,Drama,NaN,D,NaN,NaN


In [ ]:
import pickle
actorFq = open("actorFq.pkl", "rb")
actorfq = pickle.load(actorFq)
actorFq.close()

directorFq = open("directorFq.pkl", "rb")
directorfq = pickle.load(directorFq)
directorFq.close()


In [ ]:
print(actorfq)
print(directorfq)

{'Andrew Lawrence': 1, "Rickey D'Shon Collins": 1, 'Pamela Adlon': 1, 'Ashley Johnson': 2, 'Jason Davis': 2, 'Courtland Mead': 3, 'April Winchell': 4, 'Dabney Coleman': 4, "Ryan O'Donohue": 3, 'Robert Goulet': 2, 'Melissa Joan Hart': 1, 'Peter MacNicol': 1, 'Tony Jay': 5, 'James Woods': 5, 'Andrea Martin': 1, 'Tress MacNeille': 2, 'Nicholas Turturro': 2, 'Kevin Michael Richardson': 4, 'Dan Castellaneta': 3, 'Diedrich Bader': 2, 'Kath Soucie': 10, 'Charles Kimbrough': 2, 'Robert Stack': 1, 'Ken Swofford': 1, 'Francesca Marie Smith': 2, 'Erik von Detten': 4, 'Paul Willson': 1, 'Patrick Renna': 1, 'Anndi McAfee': 1, 'Lane Toran': 1, 'Rachel Crane': 1, 'Klee Bragger': 2, 'R. Lee Ermey': 5, 'Clyde Kusatsu': 4, 'Clancy Brown': 4, 'Allyce Beasley': 1, 'Gregg Berger': 3, 'Elizabeth Daily': 3, 'Ron Glass': 1, 'Mark Robert Myers': 1, 'Phil Proctor': 15, 'Justin Shenkarow': 1, 'Michael Shulman': 1, 'Katey Sagal': 2, 'Emilio Estevez': 6, 'Kathryn Erbe': 2, 'Michael Tucker': 2, 'Jan Rubeš': 4, 'Car

In [ ]:
print(directorfq["Wolfgang Reitherman"])

5


In [ ]:
newDirector = dict()
def directorEncoding():
  for index, row in my_data.iterrows():    
    if row["director"] in directorfq.keys():
      continue
    else:
        newDirector[row["director"]]=0

  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,5]):
      continue
    elif my_data.iloc[idx,5] in directorfq.keys():
     my_data.at[idx,"director"]=directorfq[ my_data.at[idx,"director"]]
    else:
       my_data.at[idx,"director"]=newDirector[ my_data.at[idx,"director"]]

In [ ]:
directorEncoding()

In [ ]:
ratingCols = open("ratingCols.pkl", "rb")
ratingcols = pickle.load(ratingCols)
ratingCols.close()

otherCols = open("otherRatingCols.pkl", "rb")
otherRatingcols = pickle.load(otherCols)
otherCols.close()


In [ ]:
print(ratingcols)
print(otherRatingcols)

['PG', 'PG-13']
['Not Rated', 'G', 'R']


In [ ]:
genreCols = open("genreCols.pkl", "rb")
genrecols = pickle.load(genreCols)
genreCols.close()

otherGenreCols = open("otherGenreCols.pkl", "rb")
otherGenrecols = pickle.load(otherGenreCols)
otherGenreCols.close()


In [ ]:
print(genrecols)
print(otherGenrecols)

['Comedy', 'Adventure', 'Drama']
['Concert/Performance', 'Black Comedy', 'Horror', 'Western', 'Musical', 'Documentary', 'Thriller/Suspense', 'Romantic Comedy', 'Action']


In [ ]:
def ratingEncoding():
  for l in ratingcols:
    my_data[l] = np.where(my_data['MPAA_rating']==l,1,0)

  my_data["other_rating"] = " "
  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,3]):
      my_data["other_rating"][idx]=0
    if my_data["MPAA_rating"][idx] in otherRatingcols:
      my_data["other_rating"][idx]=1
    else:
      my_data["other_rating"][idx]=0

In [ ]:
ratingEncoding()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
my_data

,movie_title,release_date,genre,MPAA_rating,MovieSuccessLevel,director,voice-actor,PG,PG-13,other_rating
0,Washington Square,1997,Drama,PG,D,NaN,NaN,1,0,0
1,Million Dollar Arm,2014,Drama,PG,C,NaN,NaN,1,0,0
2,The Rookie,2002,Drama,G,B,NaN,NaN,0,0,1
3,Betsy's Wedding,1990,Comedy,R,C,NaN,NaN,0,0,1
4,Tombstone,1993,Western,R,B,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...
108,Innocent Blood,1992,Horror,R,D,NaN,NaN,0,0,1
109,One Good Cop,1991,Action,R,C,NaN,NaN,0,0,1
110,Doug's 1st Movie,1999,Adventure,G,C,NaN,NaN,0,0,1
111,Wild Hearts Can't Be Broken,1991,Drama,NaN,D,NaN,NaN,0,0,0


In [ ]:
def genreEncoding():
  for l in genrecols:
    my_data[l] = np.where(my_data['genre']==l,1,0)

  my_data["other_genre"] = " "
  for idx,row in my_data.iterrows():
    if pd.isna(my_data.iloc[idx,2]):
      my_data["other_genre"][idx]=0
    if my_data["genre"][idx] in otherGenrecols:
      my_data["other_genre"][idx]=1
    else:
      my_data["other_genre"][idx]=0

In [ ]:
genreEncoding()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
my_data

,movie_title,release_date,genre,MPAA_rating,MovieSuccessLevel,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,Washington Square,1997,Drama,PG,D,NaN,NaN,1,0,0,0,0,1,0
1,Million Dollar Arm,2014,Drama,PG,C,NaN,NaN,1,0,0,0,0,1,0
2,The Rookie,2002,Drama,G,B,NaN,NaN,0,0,1,0,0,1,0
3,Betsy's Wedding,1990,Comedy,R,C,NaN,NaN,0,0,1,1,0,0,0
4,Tombstone,1993,Western,R,B,NaN,NaN,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Innocent Blood,1992,Horror,R,D,NaN,NaN,0,0,1,0,0,0,1
109,One Good Cop,1991,Action,R,C,NaN,NaN,0,0,1,0,0,0,1
110,Doug's 1st Movie,1999,Adventure,G,C,NaN,NaN,0,0,1,0,1,0,0
111,Wild Hearts Can't Be Broken,1991,Drama,NaN,D,NaN,NaN,0,0,0,0,0,1,0


In [ ]:
my_data= my_data.drop("genre",axis = 1)
my_data= my_data.drop("MPAA_rating",axis = 1)


In [ ]:
my_data

,movie_title,release_date,MovieSuccessLevel,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,Washington Square,1997,D,NaN,NaN,1,0,0,0,0,1,0
1,Million Dollar Arm,2014,C,NaN,NaN,1,0,0,0,0,1,0
2,The Rookie,2002,B,NaN,NaN,0,0,1,0,0,1,0
3,Betsy's Wedding,1990,C,NaN,NaN,0,0,1,1,0,0,0
4,Tombstone,1993,B,NaN,NaN,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
108,Innocent Blood,1992,D,NaN,NaN,0,0,1,0,0,0,1
109,One Good Cop,1991,C,NaN,NaN,0,0,1,0,0,0,1
110,Doug's 1st Movie,1999,C,NaN,NaN,0,0,1,0,1,0,0
111,Wild Hearts Can't Be Broken,1991,D,NaN,NaN,0,0,0,0,0,1,0


In [ ]:
newActors = dict()
def actorEncoding():
  for index, row in my_data.iterrows(): 
    if  type  (row['voice-actor'])is list:
      for actor in row['voice-actor']:
        if actor in actorfq.keys():
          continue
        else:     
            newActors[actor] = 0

  for index, row in my_data.iterrows():
    if  type(row['voice-actor'])is list:
      fq = 0
      for actor in row['voice-actor']:
        if actor in actorfq.keys():
          fq = fq + actorfq[actor]
          my_data.at[index,"voice-actor"] = fq
        else:
          fq = fq + newActors[actor]
          my_data.at[index,"voice-actor"] = fq

In [ ]:
actorEncoding()

In [ ]:
my_data

,movie_title,release_date,MovieSuccessLevel,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,Washington Square,1997,D,NaN,NaN,1,0,0,0,0,1,0
1,Million Dollar Arm,2014,C,NaN,NaN,1,0,0,0,0,1,0
2,The Rookie,2002,B,NaN,NaN,0,0,1,0,0,1,0
3,Betsy's Wedding,1990,C,NaN,NaN,0,0,1,1,0,0,0
4,Tombstone,1993,B,NaN,NaN,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
108,Innocent Blood,1992,D,NaN,NaN,0,0,1,0,0,0,1
109,One Good Cop,1991,C,NaN,NaN,0,0,1,0,0,0,1
110,Doug's 1st Movie,1999,C,NaN,NaN,0,0,1,0,1,0,0
111,Wild Hearts Can't Be Broken,1991,D,NaN,NaN,0,0,0,0,0,1,0


In [ ]:
my_data= my_data.drop("movie_title",axis = 1)

In [ ]:
label = my_data["MovieSuccessLevel"]
my_data = my_data.drop("MovieSuccessLevel",axis = 1)

In [ ]:
my_data

,release_date,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,1997,NaN,NaN,1,0,0,0,0,1,0
1,2014,NaN,NaN,1,0,0,0,0,1,0
2,2002,NaN,NaN,0,0,1,0,0,1,0
3,1990,NaN,NaN,0,0,1,1,0,0,0
4,1993,NaN,NaN,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
108,1992,NaN,NaN,0,0,1,0,0,0,1
109,1991,NaN,NaN,0,0,1,0,0,0,1
110,1999,NaN,NaN,0,0,1,0,1,0,0
111,1991,NaN,NaN,0,0,0,0,0,1,0


In [ ]:
knnimputerFile = open("knnimputer.pkl", "rb")
knnimputer = pickle.load(knnimputerFile)
knnimputerFile.close()


In [ ]:
def dataImputation(my_data):
  Xtrans = knnimputer.transform(my_data)
  my_data = pd.DataFrame(Xtrans, columns=my_data.columns)
  for col in my_data.columns:
    my_data[col] = my_data[col].astype(int)
  return my_data
my_data = dataImputation(my_data)

In [ ]:
def labelTransform(label):
  success = {"S": 5, "A" : 4, "B":3,"C":2,"D":1}
  for i in range(len(label)):
    label[i] = success[label[i]]
  label = label.astype(int)
  return label
label=labelTransform(label)

In [ ]:
x = my_data

In [ ]:
droppedColsFile = open("droppedCols.pkl", "rb")
droppedcols = pickle.load(droppedColsFile)
droppedColsFile.close()


In [ ]:
droppedcols

['PG-13', 'other_rating', 'Comedy', 'other_genre']

In [ ]:
for col in droppedcols:
  x = x.drop(col, axis = 1)

In [ ]:
x

,release_date,director,voice-actor,PG,Adventure,Drama
0,1997,3,43,1,0,1
1,2014,1,69,1,0,1
2,2002,1,45,0,0,1
3,1990,2,44,0,0,0
4,1993,1,31,0,0,0
...,...,...,...,...,...,...
108,1992,2,101,0,0,0
109,1991,2,92,0,0,0
110,1999,2,212,0,1,0
111,1991,1,29,0,0,1


In [ ]:
label

0      1
1      2
2      3
3      2
4      3
      ..
108    1
109    2
110    2
111    1
112    2
Name: MovieSuccessLevel, Length: 113, dtype: int64

In [ ]:
model = open("LogisticRegression.sav", "rb")
logistic  = pickle.load(model)
model.close()
predictions = logistic.predict(x)
accuracy=np.mean(predictions == label)*100
print(accuracy)

38.93805309734513


In [ ]:
model = open("linearSVM.sav", "rb")
linearSVM  = pickle.load(model)
model.close()
predictions = linearSVM.predict(x)
accuracy=np.mean(predictions == label)*100
print(accuracy)

36.283185840707965


In [ ]:
model = open("rfc.sav", "rb")
rfc = pickle.load(model)
model.close()
predictions = rfc.predict(x)
accuracy=np.mean(predictions == label)*100
print(accuracy)

38.05309734513274


In [ ]:
model = open("bdt.sav", "rb")
bdt = pickle.load(model)
model.close()
predictions = bdt.predict(x)
accuracy=np.mean(predictions == label)*100
print(accuracy)

32.743362831858406
